In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [3]:
conda install -c conda-forge dlib

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\ProgramData\Anaconda3

  added / updated specs:
    - dlib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _anaconda_depends-2022.10  |           py39_2          68 KB
    anaconda-custom            |           py39_1           4 KB
    ca-certificates-2023.5.7   |       h56e8100_0         145 KB  conda-forge
    certifi-2023.5.7           |     pyhd8ed1ab_0         149 KB  conda-forge
    conda-22.9.0               |   py39hcbf5309_2         985 KB  conda-forge
    decorator-5.1.1            |     pyhd8ed1ab_0          12 KB  conda-forge
    dlib-19.24.0               |   py39hf850


The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/win-64::anaconda==2022.10=py39_0
  - defaults/win-64::holoviews==1.15.0=py39haa95532_0
  - defaults/win-64::hvplot==0.8.0=py39haa95532_0
  - defaults/win-64::ipykernel==6.15.2=py39haa95532_0
  - defaults/win-64::ipython==7.31.1=py39haa95532_1
  - defaults/noarch::ipywidgets==7.6.5=pyhd3eb1b0_1
  - defaults/win-64::jupyter==1.0.0=py39haa95532_8
  - defaults/win-64::jupyterlab==3.4.4=py39haa95532_0
  - defaults/noarch::jupyter_console==6.4.3=pyhd3eb1b0_0
  - defaults/noarch::nbclassic==0.3.5=pyhd3eb1b0_0
  - defaults/win-64::notebook==6.4.12=py39haa95532_0
  - defaults/noarch::qtconsole==5.2.2=pyhd3eb1b0_0
  - defaults/win-64::spyder==5.2.2=py39haa95532_1
  - defaults/win-64::spyder-kernels==2.2.1=py39haa95532_0
  - defaults/win-64::widgetsnbextension==3.5.2=py39haa95532_0
  - defaults/win-64::_ipyw_jlab_nb_ext_conf==0.1.0=py39haa95532_0

The envir

In [4]:
pip install face_recognition

     -------------------------------------- 100.1/100.1 MB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=c60c7644c6afe4d929ce53cb8527e409d068742be5ddd1896ac04ba382c5b7a6
  Stored in directory: c:\users\student\appdata\local\pip\cache\wheels\22\a8\60\4a2aeb763d63f50190f4c4e07069a22245347eeafdb3a67551
Successfully built face-recognition-models
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install imutils

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25847 sha256=db12edf334469e35bb4bb869a8547ee4254be2b80c22c717e31e146c3f4dbb8e
  Stored in directory: c:\users\student\appdata\local\pip\cache\wheels\4b\a5\2d\4a070a801d3a3d93f033d3ee9728f470f514826e89952df3ea
Successfully built imutils
Note: you may need to restart the kernel to use updated packages.


from imutils import paths    #working for the path
import face_recognition
import pickle      #save and open file
import cv2
import os


imagePath = list(paths.list_images('C:\\Users\\STUDENT\\Downloads\\faces'))
knownEncodings = []
knownNames = []

for (i, imagePath) in enumerate(imagePath):
    name = imagePath.split(os.path.sep)[-2]
    image = cv2.imread(imagePath)
    rgb= cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    boxes = face_recognition.face_locations(rgb,model='hog')
    encodings = face_recognition.face_encodings(rgb, boxes)
    for encoding in encodings:
        knownEncodings.append(encoding)
        knownNames.append(name)

data = {"encodings": knownEncodings, "names": knownNames}
print(data)
f = open("face_enc", "wb")
f.write(pickle.dumps(data))
print("trained")                                                                                   

In [ ]:
import face_recognition
import pickle
import time
import cv2
import os


cascPathface = os.path.dirname(cv2.__file__)+"/data/haarcascade_frontalface_alt2.xml"
faceCascade = cv2.CascadeClassifier(cascPathface)
data = pickle.loads(open('face_enc',"rb").read())
print("Streaming started")
video_capture = cv2.VideoCapture(0)

while True:
    ret, frame=video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE)
   
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    encodings = face_recognition.face_encodings(rgb)
    names = []
   
    for encoding in encodings:
        matches = face_recognition.compare_faces(data["encodings"],encoding);
        name = "Unknown"
        if True in matches:
            matchedIdxs= [i for (i, b)in enumerate(matches) if b]
            counts = {}
            for i in matchedIdxs:
                name = data["names"][i]
                counts[name] = counts.get(name,0) + 1
 
            name = max(counts,key=counts.get)
         
           
        names.append(name)
        for((x, y, w, h),name) in zip(faces,names):
            cv2.rectangle(frame, (x, y),(x + w, y + h),(0, 255, 0),2)
            cv2.putText(frame, name, (x, y), cv2.FONT_HERSHEY_SIMPLEX,0.75, (0, 255, 0), 2)
    cv2.imshow("preview",frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()
f.close()

Streaming started
